In [1]:
import os
import numpy as np
import keras as k
import pandas as pd
import string
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import KeyedVectors
from keras.utils import to_categorical
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nano\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Load data

In [2]:
files_path = './data/'
train_path = files_path + 'train.tsv'
test_path = files_path + 'test.tsv'
val_path = files_path + 'valid.tsv'
 p
columns =['id', 'label','statement', 'subject', 'speaker', 'job', 'state', 'party', 
          'barely_true_c', 'false_c', 'half_true_c', 'mostly_true_c', 'pants_on_fire_c',
          'venue']
l2i = {'pants-fire':0, 'false':1, 'barely-true':2, 'half-true':3, 'mostly-true':4, 'true':5}

df_train = pd.read_table(train_path, names = columns)
df_test = pd.read_table(test_path, names = columns)
df_val = pd.read_table(val_path, names = columns)

# Labels to ints
df_train['label'] = df_train['label'].map(l2i)
df_test['label'] = df_test['label'].map(l2i)
df_val['label'] = df_val['label'].map(l2i)

### Meta

In [3]:
def parse_meta(df):
    cols = ['subject', 'speaker', 'job', 'state', 'party', 'venue']
    for c in cols:
        df[c] = df[c].apply(lambda x: "".join([z for z in [y.lower() for y in str(x).translate(str.maketrans('', '', string.punctuation+string.digits+'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'')).split(" ")] if z not in stopwords.words('english')]))
    return df

In [4]:
df_train = pd.read_table(train_path, names = columns)
df_test = pd.read_table(test_path, names = columns)
df_val = pd.read_table(val_path, names = columns)

# Labels to ints
df_train['label'] = df_train['label'].map(l2i)
df_test['label'] = df_test['label'].map(l2i)
df_val['label'] = df_val['label'].map(l2i)

# df_test.head(2)

In [5]:
parse_meta(df_train)
parse_meta(df_val)
parse_meta(df_test)

,id,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue
0,11972.json,5,Building a wall on the U.S.-Mexico border will...,immigration,rickperry,governor,texas,republican,30,30,42,23,18,radiointerview
1,11685.json,1,Wisconsin is on pace to double the number of l...,jobs,katrinashankland,staterepresentative,wisconsin,democrat,2,1,0,0,0,newsconference
2,11096.json,1,Says John McCain has done nothing to help the ...,militaryveteransvotingrecord,donaldtrump,presidentelect,newyork,republican,63,114,51,37,61,commentsabcsweek
3,5209.json,3,Suzanne Bonamici supports a plan that will cut...,medicaremessagemachinecampaignadvertising,robcornilles,consultant,oregon,republican,1,1,3,1,1,radioshow
4,9524.json,0,When asked by a reporter whether hes at the ce...,campaignfinancelegalissuescampaignadvertising,statedemocraticpartywisconsin,nan,wisconsin,democrat,5,7,2,2,7,webvideo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,7334.json,3,Says his budget provides the highest state fun...,education,rickscott,governor,florida,republican,28,23,38,34,7,newsconference
1263,9788.json,2,Ive been here almost every day.,civilrightscrimecriminaljustice,jaynixon,governor,missouri,democrat,2,0,0,1,0,abcsweek
1264,10710.json,2,"In the early 1980s, Sen. Edward Kennedy secret...",bipartisanshipcongressforeignpolicyhistory,mackubinthomasowens,seniorfellowforeignpolicyresearchinstitute,rhodeisland,columnist,1,0,0,0,0,commentaryprovidencejournal
1265,3186.json,2,Says an EPA permit languished under Strickland...,environmentgovernmentefficiency,johnkasich,governorohiojan,ohio,republican,9,8,10,18,3,newsconference


In [6]:
t = Tokenizer(oov_token=True)
cols = ['subject', 'speaker', 'job', 'state', 'party', 'venue']
dfs = [df_train, df_val, df_test]
ts = []
for c in cols:
    ts.append(Tokenizer(oov_token=True))
for d in dfs:
    for i, c in enumerate(cols):
        ts[i].fit_on_texts(d[c])

In [7]:
def count_t_index(ts):
    for t in ts:
        print(len(t.word_index))

In [8]:
def parse_statement(df):
    df['p_statement'] = df['statement'].apply(lambda x: [z for z in [y.lower() for y in x.translate(str.maketrans('', '', string.punctuation+string.digits+'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'')).split(" ")] if z not in stopwords.words('english')])
    
    df['length'] = df['p_statement'].apply(lambda x: len(x))
    return df

In [9]:
parse_statement(df_train)
parse_statement(df_val)
parse_statement(df_test)

,id,label,statement,subject,speaker,job,state,party,barely_true_c,false_c,half_true_c,mostly_true_c,pants_on_fire_c,venue,p_statement,length
0,11972.json,5,Building a wall on the U.S.-Mexico border will...,immigration,rickperry,governor,texas,republican,30,30,42,23,18,radiointerview,"[building, wall, usmexico, border, take, liter...",7
1,11685.json,1,Wisconsin is on pace to double the number of l...,jobs,katrinashankland,staterepresentative,wisconsin,democrat,2,1,0,0,0,newsconference,"[wisconsin, pace, double, number, layoffs, year]",6
2,11096.json,1,Says John McCain has done nothing to help the ...,militaryveteransvotingrecord,donaldtrump,presidentelect,newyork,republican,63,114,51,37,61,commentsabcsweek,"[says, john, mccain, done, nothing, help, vets]",7
3,5209.json,3,Suzanne Bonamici supports a plan that will cut...,medicaremessagemachinecampaignadvertising,robcornilles,consultant,oregon,republican,1,1,3,1,1,radioshow,"[suzanne, bonamici, supports, plan, cut, choic...",9
4,9524.json,0,When asked by a reporter whether hes at the ce...,campaignfinancelegalissuescampaignadvertising,statedemocraticpartywisconsin,nan,wisconsin,democrat,5,7,2,2,7,webvideo,"[asked, reporter, whether, hes, center, crimin...",15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262,7334.json,3,Says his budget provides the highest state fun...,education,rickscott,governor,florida,republican,28,23,38,34,7,newsconference,"[says, budget, provides, highest, state, fundi...",9
1263,9788.json,2,Ive been here almost every day.,civilrightscrimecriminaljustice,jaynixon,governor,missouri,democrat,2,0,0,1,0,abcsweek,"[ive, almost, every, day]",4
1264,10710.json,2,"In the early 1980s, Sen. Edward Kennedy secret...",bipartisanshipcongressforeignpolicyhistory,mackubinthomasowens,seniorfellowforeignpolicyresearchinstitute,rhodeisland,columnist,1,0,0,0,0,commentaryprovidencejournal,"[early, sen, edward, kennedy, secretly, offere...",15
1265,3186.json,2,Says an EPA permit languished under Strickland...,environmentgovernmentefficiency,johnkasich,governorohiojan,ohio,republican,9,8,10,18,3,newsconference,"[says, epa, permit, languished, strickland, ne...",12


In [10]:
print(np.median(df_train['length']))
print(np.median(df_val['length']))
print(np.median(df_test['length']))

10.0
10.0
10.0


In [11]:
print(np.mean(df_train['length']))
print(np.mean(df_val['length']))
print(np.mean(df_test['length']))

11.10615234375
11.092679127725857
11.304656669297554


In [12]:
t = Tokenizer()

t.fit_on_texts(df_test['p_statement'])
t.fit_on_texts(df_val['p_statement'])
t.fit_on_texts(df_test['p_statement'])

In [13]:
len(t.word_index)

6100

In [14]:
import pickle
def save_tokenizer(tokenizer, filename='tokenizer.pickle'):
    with open(filename, 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
def load_tokenizer(filename='tokenizer.pickle'):
    with open(filename, 'rb') as handle:
        tokenizer = pickle.load(handle)
    return tokenizer
def save_tokenizers(ts, filename='tokenizer'):
    for i, t in enumerate(ts):
        save_tokenizer(t, filename=filename+"_"+str(i)+".pickle")
def load_tokenizers(count=6, filename='tokenizer'):
    ts = []
    for i in range(count):
        ts.append(load_tokenizer(filename=filename+"_"+str(i)+".pickle"))
    return ts
def load_df(filename):
    return pd.read_pickle(filename)
def load_embedding_matrix(filename = 'embedding_matrix.npy'):
    return np.load(filename)

In [15]:
save_tokenizer(t)
save_tokenizers(ts)
df_train.to_pickle('df_train.pkl')
df_val.to_pickle('df_val.pkl')
df_test.to_pickle('df_test.pkl')

In [16]:
tss = load_tokenizers()

In [17]:
count_t_index(tss)

4496
3304
1231
69
26
4213


In [18]:
df_train = load_df('df_train.pkl')
df_val = load_df('df_val.pkl')
df_test = load_df('df_test.pkl')

In [19]:
t = load_tokenizer()

In [20]:
from gensim import models
w = models.KeyedVectors.load_word2vec_format('./data/GoogleNews-vectors-negative300.bin', binary=True)

In [21]:
dict_length = len(t.word_index) + 1

In [22]:
embedding_matrix = np.zeros((dict_length, 300))

In [23]:
for word, i in t.word_index.items():
    if word in w:
        embedding_matrix[i] = w[word]
np.save("embedding_matrix.npy", embedding_matrix)

In [24]:
matrix = load_embedding_matrix()

In [25]:
(matrix == embedding_matrix).all()

True

In [26]:
def df_to_input(df, tokenizer, max_len):
    return pad_sequences(tokenizer.texts_to_sequences(df.p_statement), maxlen=max_len, padding='post', truncating='post')
def df_to_label(df):
    return df.label.to_numpy()

In [27]:
pad_sequences(t.texts_to_sequences(df_test.p_statement), maxlen=10, padding='post', truncating='post')

array([[1371,  308, 1626, ...,    0,    0,    0],
       [  47, 2061,  613, ...,    0,    0,    0],
       [   2,  100,  129, ...,    0,    0,    0],
       ...,
       [ 717,  144, 4237, ..., 4238,  615, 4239],
       [   2,  551, 1142, ...,  694,   71,  281],
       [   2,   62,   50, ...,  104,    7,   60]])

In [28]:
statement_to_input(t, df_test.p_statement[0], 10)

NameError: name 'statement_to_input' is not defined

In [ ]:
#tokenizing the statements and creating the embedding matrix
texts=df_train.statement
NUM_WORDS=20000
tokenizer = Tokenizer(num_words=NUM_WORDS,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'',
                      lower=True)
tokenizer.fit_on_texts(texts)
sequences_train = tokenizer.texts_to_sequences(texts)
sequences_valid=tokenizer.texts_to_sequences(df_val.statement)
word_index = tokenizer.word_index
vocabulary_size=min(len(word_index)+1,NUM_WORDS)
EMBEDDING_DIM=300
print(len(word_index))

embedding_matrix = np.load('embedding_matrix.npy')

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

# Gets data from a dataframe
def get_data(dataframe, tokenizer):
    labels=dataframe.label.unique()
    dic={}
    for i,label in enumerate(labels):
        dic[label]=i
    labels=dataframe.label.apply(lambda x:dic[x])
    Y = to_categorical(np.asarray(labels[dataframe.index]))
    
    X = pad_sequences(tokenizer.texts_to_sequences(dataframe.statement))
    return X, Y

In [ ]:
x_train, y_train = get_data(df_train, tokenizer)

In [ ]:
labels = df_train.label.unique()
dic={}
for i,label in enumerate(labels):
    dic[label]=i
labels = df_train.label.apply(lambda x:dic[x])
y_train = to_categorical(np.asarray(labels[df_train.index]))
y_val = to_categorical(np.asarray(labels[df_val.index]))
X_train = pad_sequences(sequences_train)
X_val = pad_sequences(sequences_valid,maxlen=X_train.shape[1])

print('Shape of X train and X validation tensor:', X_train.shape,X_val.shape)
print('Shape of label train and validation tensor:', y_train.shape,y_val.shape)
